## Sensor Noise Suppression using Multivariate Regression

### Import the dependencies

In [1]:
%load_ext lab_black

# Common Imports
import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer

from sklearn.decomposition import PCA

# Assessing performance
from sklearn.model_selection import GridSearchCV

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

# Making a prediction with the direct multioutput regression model
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc("axes", labelsize=14)
mpl.rc("xtick", labelsize=12)
mpl.rc("ytick", labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings

warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore", category=DeprecationWarning)
warnings.filterwarnings(action="ignore", category=FutureWarning)

# To make this notebook's output identical at every run
np.random.seed(42)

### Preprocess the data

In [2]:
data = (
    pd.read_csv("position_log_v2.csv")
    .query("marker != 8")  # too many null values for tag 1
    .query("marker % 2 == 0")
    .drop(["x", "y", "z", "anchors", "time"], axis=1)
    .reset_index(drop=True)
)

data["tag"] = data["tag"].replace(52630, 3)

data["marker"] = data["marker"].map(
    {j: i + 1 for i, j in enumerate(data.marker.unique())}
)

data.columns = [
    "position",
    "tag",
    "anchor_1",
    "anchor_2",
    "anchor_3",
    "anchor_4",
    "anchor_5",
]

data

,position,tag,anchor_1,anchor_2,anchor_3,anchor_4,anchor_5
0,1,1,204.0,871.0,1072.0,375.0,820.0
1,1,2,209.0,871.0,1082.0,468.0,820.0
2,1,3,189.0,882.0,1117.0,401.0,817.0
3,1,1,205.0,872.0,1072.0,375.0,824.0
4,1,2,214.0,875.0,1082.0,401.0,827.0
...,...,...,...,...,...,...,...
27655,11,2,336.0,602.0,814.0,160.0,603.0
27656,11,3,327.0,636.0,837.0,240.0,615.0
27657,11,1,332.0,618.0,801.0,184.0,586.0
27658,11,2,338.0,603.0,812.0,159.0,599.0


In [3]:
position = data.query("tag == 1").loc[:, ["position"]].reset_index(drop=True)

data_tag_1 = (
    data.query("tag == 1")
    .drop(["position", "tag"], axis=1)
    .reset_index(drop=True)
    .fillna(0)
)

data_tag_2 = (
    data.query("tag == 2")
    .drop(["position", "tag"], axis=1)
    .reset_index(drop=True)
    .fillna(0)
)

data_tag_3 = (
    data.query("tag == 3")
    .drop(["position", "tag"], axis=1)
    .reset_index(drop=True)
    .fillna(0)
)

### Compute distance from A to B, B to C, and A to C

$d(A,B) = \sqrt{(A_1-B_1)^2 + (A_2-B_2)^2 + \dots + (A_n-B_n)^2}$

In [4]:
AB = data_tag_1.sub(data_tag_2).pow(2).sum(axis=1).pow(0.5)
BC = data_tag_2.sub(data_tag_3).pow(2).sum(axis=1).pow(0.5)
AC = data_tag_1.sub(data_tag_3).pow(2).sum(axis=1).pow(0.5)

pd.DataFrame(data=[AB, BC, AC], index=["AB", "BC", "AC"]).T

,AB,BC,AC
0,93.669632,79.018985,55.281100
1,29.580399,44.339599,53.544374
2,43.520110,53.441557,57.480431
3,50.378567,53.962950,61.335145
4,63.812225,55.901699,60.835845
...,...,...,...
9215,40.422766,94.535708,77.890949
9216,32.619013,91.274312,75.412201
9217,41.509035,95.420124,76.105190
9218,36.510273,91.159201,77.077883


In [ ]:
data_tag_1

In [ ]:
data_tag_2

In [ ]:
np.abs(AB - BC)

In [ ]:
np.abs(AB - AC)

In [ ]:
np.abs(AC - BC)

### Join the data

In [5]:
data_tag_1.columns = map("{}_tag_1".format, data_tag_1.columns)
data_tag_2.columns = map("{}_tag_2".format, data_tag_2.columns)
data_tag_3.columns = map("{}_tag_3".format, data_tag_3.columns)

data = position.join([data_tag_1, data_tag_2, data_tag_3])
data

,position,anchor_1_tag_1,anchor_2_tag_1,anchor_3_tag_1,anchor_4_tag_1,anchor_5_tag_1,anchor_1_tag_2,anchor_2_tag_2,anchor_3_tag_2,anchor_4_tag_2,anchor_5_tag_2,anchor_1_tag_3,anchor_2_tag_3,anchor_3_tag_3,anchor_4_tag_3,anchor_5_tag_3
0,1,204.0,871.0,1072.0,375.0,820.0,209.0,871.0,1082.0,468.0,820.0,189.0,882.0,1117.0,401.0,817.0
1,1,205.0,872.0,1072.0,375.0,824.0,214.0,875.0,1082.0,401.0,827.0,188.0,880.0,1116.0,398.0,817.0
2,1,206.0,870.0,1074.0,374.0,822.0,208.0,874.0,1078.0,417.0,819.0,187.0,885.0,1121.0,396.0,817.0
3,1,203.0,872.0,1069.0,373.0,821.0,211.0,873.0,1082.0,421.0,821.0,190.0,890.0,1121.0,396.0,815.0
4,1,199.0,870.0,1070.0,371.0,829.0,207.0,872.0,1082.0,433.0,825.0,188.0,880.0,1120.0,399.0,815.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9215,11,333.0,622.0,801.0,183.0,585.0,334.0,602.0,815.0,157.0,604.0,328.0,636.0,832.0,241.0,624.0
9216,11,333.0,617.0,802.0,184.0,584.0,335.0,605.0,814.0,160.0,598.0,326.0,634.0,833.0,242.0,616.0
9217,11,330.0,621.0,807.0,182.0,581.0,337.0,604.0,812.0,154.0,605.0,327.0,634.0,834.0,240.0,620.0
9218,11,329.0,618.0,802.0,182.0,583.0,336.0,602.0,814.0,160.0,603.0,327.0,636.0,837.0,240.0,615.0


In [6]:
data.isnull().sum()

position          0
anchor_1_tag_1    0
anchor_2_tag_1    0
anchor_3_tag_1    0
anchor_4_tag_1    0
anchor_5_tag_1    0
anchor_1_tag_2    0
anchor_2_tag_2    0
anchor_3_tag_2    0
anchor_4_tag_2    0
anchor_5_tag_2    0
anchor_1_tag_3    0
anchor_2_tag_3    0
anchor_3_tag_3    0
anchor_4_tag_3    0
anchor_5_tag_3    0
dtype: int64

In [7]:
# Amount of readings per location
data["position"].value_counts()

8     1026
7      911
5      911
9      911
3      910
2      798
1      798
6      796
11     796
10     796
4      567
Name: position, dtype: int64

### LOOCV for Evaluating Machine Learning Algorithms

In [19]:
X_tx_pos = [1597, 766, 530, 839, 1269, 530, 1597, 530, 1597, 944, 1315]
y_pos = [1958, 1690, 2040, 2244, 1744, 1690, 1690, 2302, 2302, 2083, 1925]

for i in sorted(data["position"].astype(int).unique()):

    #####################################################################
    # X_train - train on 11/12
    #####################################################################

    # leave one position out
    training_data = data.query(f"position != {i}").reset_index(drop=True)
    position = training_data[["position"]].copy()
    X_train = training_data.drop("position", axis=1)

    # normalize the 10 positions
    min_max_scaler = MinMaxScaler()
    min_max_scaler.fit(X_train)
    scaled_train_features = min_max_scaler.transform(X_train)

    # convert the normalized matrix to a dataframe
    X_train = pd.DataFrame(
        scaled_train_features, index=X_train.index, columns=X_train.columns
    )

    # add the position each row corresponds to
    X_train = position.join(X_train)

    X_train_transposed = pd.DataFrame()

    for j in X_train["position"].unique():

        # impute the null rows with the mean (fill 0 where this is not possible)
        X_train.loc[X_train["position"] == j] = (
            X_train.query(f"position == {j}")
            .fillna(X_train.query(f"position == {j}").mean())
            .fillna(0)
        )

        # stack the position data into a single row
        df = (
            X_train.query(f"position == {j}")
            .copy(deep=True)
            .drop("position", axis=1)
            .reset_index(drop=True)
        )

        df.index = df.index + 1
        df_out = df.stack()
        df_out.index = df_out.index.map("{0[1]}_{0[0]}".format)
        df_out = df_out.to_frame().T

        # compose everything in a single dateframe
        X_train_transposed = (
            X_train_transposed.append(df_out).fillna(0).reset_index(drop=True)
        )

    # match each row with its corresponding position in the Y_train
    x = x_pos.copy()
    y = y_pos.copy()

    x_test_pos = x.pop(i - 1)
    y_test_pos = y.pop(i - 1)

    Y_train = pd.DataFrame()
    Y_train["x"] = x
    Y_train["y"] = y

    ####################################################################
    # X_test - test on 1/12
    ####################################################################

    testing_set = data.query(f"position == {i}").reset_index(drop=True)
    position = testing_set[["position"]].copy()

    X_test = testing_set.drop("position", axis=1)

    scaled_test_features = min_max_scaler.transform(X_test)

    X_test = pd.DataFrame(
        scaled_test_features, index=X_test.index, columns=X_test.columns
    )

    X_test = position.join(X_test)

    X_test.loc[X_test["position"] == i] = (
        X_test.query(f"position == {i}")
        .fillna(X_test.query(f"position == {i}").mean())
        .fillna(0)
    )

    df = X_test.drop("position", axis=1).reset_index(drop=True)

    df.index = df.index + 1
    df_out = df.stack()
    df_out.index = df_out.index.map("{0[1]}_{0[0]}".format)
    X_test_transposed = df_out.to_frame().T

    if X_train_transposed.shape[1] >= X_test_transposed.shape[1]:
        X_test_transposed = (
            X_train_transposed.append(X_test_transposed).fillna(0).tail(1)
        )
    else:
        X_train_transposed = (
            X_test_transposed.append(X_train_transposed).fillna(0).tail(10)
        )

    Y_test = pd.DataFrame()
    Y_test["x"] = [x_test_pos]
    Y_test["y"] = [y_test_pos]

    #####################################################################
    # PCA
    #####################################################################

    #     pca = PCA()
    #     pca.fit(X_train_transposed)

    #     X_train = pca.transform(X_train_transposed)
    #     X_test = pca.transform(X_test_transposed)

    X_train = X_train_transposed
    X_test = X_test_transposed

    #     print("Shape of X_train:", X_train.shape)
    #     print("Shape of X_test: ", X_test.shape)

    #####################################################################
    # Train a non-linear SVM
    #####################################################################

    model = SVR()
    wrapper = MultiOutputRegressor(model)
    wrapper.fit(X_train, Y_train)

    yhat = wrapper.predict(X_test)

    print("Actual:", Y_test.to_numpy()[0])
    print("Predicted:", yhat[0].tolist())
    print("-" * 50)

Actual: [1597 1958]
Predicted: [893.1733019167767, 1982.8089016340691]
--------------------------------------------------
Actual: [ 766 1690]
Predicted: [1104.8786686490844, 1999.218734076037]
--------------------------------------------------
Actual: [ 530 2040]
Predicted: [1105.815975285063, 1940.0882030183516]
--------------------------------------------------
Actual: [ 839 2244]
Predicted: [1105.41238473248, 1941.2017145960729]
--------------------------------------------------
Actual: [1269 1744]
Predicted: [893.216865020605, 1999.3317434307025]
--------------------------------------------------
Actual: [ 530 1690]
Predicted: [1104.8964692631828, 1999.239789038415]
--------------------------------------------------
Actual: [1597 1690]
Predicted: [890.7238795579365, 1999.9486486778835]
--------------------------------------------------
Actual: [ 530 2302]
Predicted: [1107.27391970097, 1941.7589924505696]
--------------------------------------------------
Actual: [1597 2302]
Predict

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

fs = SelectKBest(score_func=f_regression, k=10)
X_selected = fs.fit_transform(X_train_transposed, pos_coord)
X_selected.shape

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10).fit_transform(X_train_transposed)
pca.shape

In [ ]:
pca

In [ ]:
df = (
    training_data.query("position == 5")
    .drop(["position", "x", "y"], axis=1)
    .reset_index(drop=True)
    .fillna(0)
)

df.index = df.index + 1
df_out = df.stack()
df_out.index = df_out.index.map("{0[1]}_{0[0]}".format)
pos_5 = df_out.to_frame().T
pos_5

In [ ]:
# the problem is that the input vector needs to be 15 or does it not?
# it makes sense that the longer you stay the less sparse the matrix is going to be -- resulting in better accracy
# the input vector, thus, does not need to be of length 15
pos_5.append(pos_1).reset_index(drop=True)

In [ ]:
for i in X_train["position"].unique():
    print(i, X_train.query("position == {}".format(i)).isnull().any(axis=1).sum())

In [ ]:
X_train = data.drop(["position", "x", "y"], axis=1)
X_train

In [ ]:
# Normalization
min_max_scaler = MinMaxScaler()
min_max_scaled_features = min_max_scaler.fit_transform(X_train)

In [ ]:
X_train = pd.DataFrame(
    min_max_scaled_features, index=X_train.index, columns=X_train.columns
)
X_train = position.join(X_train)
X_train

In [ ]:
for i in X_train["position"].unique():
    X_train.loc[X_train["position"] == i] = X_train.query(
        "position == {}".format(i)
    ).fillna(X_train.query("position == {}".format(i)).mean())

In [ ]:
for i in X_train["position"].unique():
    print(i, X_train.query("position == {}".format(i)).isnull().any(axis=1).sum())

In [ ]:
# Final X_train
X_train = X_train.drop(["position"], axis=1)
X_train

In [ ]:
y_train = data[["x", "y"]].copy()
y_train

### Train the models

In [ ]:
row = [
    0.004216,
    0.837327,
    0.920259,
    0.239286,
    0.777234,
    0.004296,
    0.839066,
    0.917794,
    0.351052,
    0.854139,
    0.004979,
    0.816065,
    0.956656,
    0.213820,
    0.802043,
]

In [ ]:
model = LinearSVR()
wrapper = MultiOutputRegressor(model)
wrapper.fit(X_train, y_train)

yhat = wrapper.predict([row])

print("Predicted:", yhat[0].round())

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

yhat = model.predict([row])

print("Predicted:", yhat[0].round())

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).fit(X_train)
pca_2d = pca.transform(X_train)

import pylab as pl
for i in range(0, pca_2d.shape[0]):
    if y_train[i] == 1:
        c1 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='r',marker='+')
    elif y_train[i] == 2:
        c2 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='g',marker='o')
    elif y_train[i] == 3:
        c3 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='b',marker='*')

pl.legend([c1, c2, c3], ['Position 1', 'Position 2', 'Position 3'])
pl.title('Dataset with 3 clusters and known outcomes')
pl.show()